# Using SQL to answer business questions

In this project, we will be using a fictional database and SQL to answer potential business questions (also fictional).

We start by connecting Jupyter Notebook to the database file `Chinook.db`.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Chinook database tables and views

In [2]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


There are 11 tables and 1 view shown above.  

## Tracks sold in the US by genre

Next, we query the number of tracks sold by genre **in the US** and their percentage of the total.

In [3]:
%%sql

SELECT
    g.name AS genre
    ,COUNT(t.track_id) AS tracks_sum
    ,ROUND((COUNT(t.track_id) / CAST((
        SELECT COUNT(*)
        FROM track t
        INNER JOIN invoice_line il ON t.track_id = il.track_id
        INNER JOIN invoice i ON il.invoice_id = i.invoice_id
        INNER JOIN customer c ON i.customer_id = c.customer_id
        WHERE c.country = "USA"
    ) AS FLOAT)), 4) AS tracks_pct
FROM genre g
INNER JOIN track t ON g.genre_id = t.genre_id
INNER JOIN invoice_line il ON t.track_id = il.track_id
INNER JOIN invoice i ON il.invoice_id = i.invoice_id
INNER JOIN customer c ON i.customer_id = c.customer_id
WHERE c.country = "USA"
GROUP BY genre
ORDER BY tracks_sum DESC
;

Done.


genre,tracks_sum,tracks_pct
Rock,561,0.5338
Alternative & Punk,130,0.1237
Metal,124,0.118
R&B/Soul,53,0.0504
Blues,36,0.0343
Alternative,35,0.0333
Latin,22,0.0209
Pop,22,0.0209
Hip Hop/Rap,20,0.019
Jazz,14,0.0133


We can see that Rock tracks sold make up more than half of the total (53.4%), followed by Alternative & Punk (12.4%) and Metal (11.8%).  

## Total sales by sales representative

Let's see how each sales representative performed by looking at their total sales amounts below.

In [4]:
%%sql

SELECT DISTINCT
    e.employee_id
    ,e.last_name || ", " || e.first_name AS ee_name
    ,ROUND(SUM(i.total), 2) AS sales_tot
FROM employee e
INNER JOIN customer c ON e.employee_id = c.support_rep_id
INNER JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY 1,2

Done.


employee_id,ee_name,sales_tot
3,"Peacock, Jane",1731.51
4,"Park, Margaret",1584.0
5,"Johnson, Steve",1393.92


There is about a 20% difference between the highest and lowest performing sales reps.  Jane Peacock leads with \$1,730 in sales, followed by Maraget Park with \$1,580, and finally Steve Johnson with \$1,390.

In [5]:
%%sql

CREATE VIEW one_cust AS
    SELECT *
    FROM customer
    GROUP BY customer_id
    HAVING COUNT(customer_id) = 1
    ;

(sqlite3.OperationalError) table one_cust already exists
[SQL: CREATE VIEW one_cust AS
    SELECT *
    FROM customer
    GROUP BY customer_id
    HAVING COUNT(customer_id) = 1
    ;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


## Various statistics by country

Next, we look at various statistics by country, such as number of customers, total sales, average customer sales, and average order amounts.

The two subqueries in the FROM clause below calculates sums/counts and groups the countries with only one customer into an "Other" category.  The outermost query then calculates sums/averages by country with the newly created "Other" category.  It also sorts the final table such that the "Other" category is the last row (even though it has the most customer counts).

In [6]:
%%sql
SELECT
country
,SUM(customers) AS customers
,ROUND(SUM(sales_tot), 2) AS sales_tot
,ROUND(CAST(sales_tot AS FLOAT) / SUM(customers), 2) AS avg_cust_sales
,ROUND(SUM(sales_tot) / SUM(invoice_id_cnt), 2) AS avg_order_amt
,CASE WHEN country = "Other" THEN 1 ELSE 0 END AS sort
FROM
    (SELECT
        CASE WHEN customers = 1 THEN "Other" ELSE country END AS country
        ,customers
        ,sales_tot
        ,invoice_id_cnt
    FROM
        (SELECT
            c.country
            ,COUNT(DISTINCT c.customer_id) AS customers
            ,SUM(il.unit_price) AS sales_tot
            ,COUNT(DISTINCT i.invoice_id) AS invoice_id_cnt
        FROM invoice_line il
        INNER JOIN invoice i ON i.invoice_id = il.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id
        GROUP BY c.country) temp)
GROUP BY country
ORDER BY sort ASC, customers DESC
;

Done.


country,customers,sales_tot,avg_cust_sales,avg_order_amt,sort
USA,13,1040.49,80.04,7.94,0
Canada,8,535.59,66.95,7.05,0
Brazil,5,427.68,85.54,7.01,0
France,5,389.07,77.81,7.78,0
Germany,4,334.62,83.66,8.16,0
United Kingdom,3,245.52,81.84,8.77,0
Czech Republic,2,273.24,136.62,9.11,0
India,2,183.15,91.58,8.72,0
Portugal,2,185.13,92.57,6.38,0
Other,15,1094.94,5.02,7.45,1


The US has the highest numbers of customers and total sales, but other countries have higher average customer sales and/or average order amounts.  For example, the Czech Republic's average customer sales is **\$137** and average order amount is **\$9.11**, compared to the US's average customer sales of **\$80** and average order amount of **\$7.94**.

Note that there are 15 countries with only one customer, which are grouped into the "Other" category at the very bottom.

## Categorizing invoices by album purchase (or not)

Finally, we collate and summarize data on album vs. single track purchases.

In [7]:
%%sql

WITH invoice_first_track AS (
    SELECT
        invoice_id
        ,MIN(track_id) AS first_track_id
    FROM invoice_line
    GROUP BY 1
    )

SELECT
    album
    ,COUNT(invoice_id) AS num_invoices
    ,ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice), 3) AS pct
FROM
    (SELECT
     ifs.*
     ,CASE WHEN (SELECT t.track_id
                 FROM track t
                 WHERE t.album_id = (SELECT t2.album_id
                                     FROM track t2
                                     WHERE t2.track_id=ifs.first_track_id)
                 
                 EXCEPT
                 
                 SELECT il2.track_id
                 FROM invoice_line il2
                 WHERE il2.invoice_id=ifs.invoice_id) IS NULL
     AND (SELECT il2.track_id
          FROM invoice_line il2
          WHERE il2.invoice_id=ifs.invoice_id
          
          EXCEPT
          
          SELECT t.track_id
          FROM track t
          WHERE t.album_id =(SELECT t2.album_id
                             FROM track t2
                             WHERE t2.track_id=ifs.first_track_id)) IS NULL
     THEN "YES" ELSE "NO" END AS album
     FROM invoice_first_track ifs)
GROUP BY album
;

Done.


album,num_invoices,pct
NO,500,0.814
YES,114,0.186


We can easily see that most invoices (over 80%) are not album purchases and instead are single track orders.